In [1]:
%pwd

'/Users/tapankheni/Data Science/Data Science Projects/Wine_Quality_Prediction/research'

In [2]:
## Changing the present working directory to create artifacts folder in the root directory
import os
os.chdir("../")
%pwd

'/Users/tapankheni/Data Science/Data Science Projects/Wine_Quality_Prediction'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [21]:
## Importing constants module to get the file paths and other necessary things
from WineQuality.constants import CONFIG_YAML_FILE_PATH, PARAMS_YAML_FILE_PATH, SCHEMA_YAML_FILE_PATH
from WineQuality.utils.common import read_yaml, create_directories

In [22]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath=CONFIG_YAML_FILE_PATH,
                 params_filepath=PARAMS_YAML_FILE_PATH, 
                 schema_filepath=SCHEMA_YAML_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


In [23]:
## Data reading phase 
import os
from urllib import request
import zipfile
from WineQuality import logger
from WineQuality.utils.common import get_size

In [24]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [25]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2024-02-16 14:44:57,240: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-02-16 14:44:57,241: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-16 14:44:57,244: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-02-16 14:44:57,245: INFO: common: created directory at: artifacts]
[2024-02-16 14:44:57,245: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-16 14:44:58,374: INFO: 2794977711: artifacts/data_ingestion/winequality.zip download! with following info 
Connection: close
Content-Length: 21984
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "85a38be3d4c70e3b6dc97fc47873623da88c9b850e1526aeb57ff236bdd2815c"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 877E:339BD:258640:363917:65CF2790
Accept-Ranges: bytes
Date: Fr